In [1]:
import sys
import numpy as np
import cv2


import os
import scipy.misc
from scipy.optimize import least_squares
import math
from copy import deepcopy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



In [2]:
def linear_estimate_3d_point(image_points, camera_matrices):
    N = image_points.shape[0]
    A = np.zeros((2*N, 4))
    A_set = []

    for i in range(N):
        pi = image_points[i]
        Mi = camera_matrices[i]
        Aix = pi[0]*Mi[2] - Mi[0]
        Aiy = pi[1]*Mi[2] - Mi[1]
        A_set.append(Aix)
        A_set.append(Aiy)

    for i in range(A.shape[0]):
        A[i] = A_set[i]

    U, s, VT = np.linalg.svd(A)
    P_homo = VT[-1]
    P_homo /= P_homo[-1]
    P = P_homo[:3]

    return P

In [3]:
def reprojection_error(point_3d, image_points, camera_matrices):
    N = image_points.shape[0]
    error_set = []
    point_3d_homo = np.hstack((point_3d, 1))

    for i in range(N):
        pi = image_points[i]
        Mi = camera_matrices[i]
        Yi = Mi.dot(point_3d_homo)
        # compute error
        pi_prime = 1.0 / Yi[2] * np.array([Yi[0], Yi[1]])
        error_i = (pi_prime - pi)
        error_set.append(error_i[0])
        error_set.append(error_i[1])

    error = np.array(error_set)
    return error

In [4]:
def jacobian(point_3d, camera_matrices):
    J = np.zeros((2*camera_matrices.shape[0], 3))
    point_3d_homo = np.hstack((point_3d, 1))
    J_set = []

    for i in range(camera_matrices.shape[0]):
        Mi = camera_matrices[i]
        pi = Mi.dot(point_3d_homo)
        Jix = (pi[2]*np.array([Mi[0, 0], Mi[0, 1], Mi[0, 2]]) \
              - pi[0]*np.array([Mi[2, 0], Mi[2, 1], Mi[2, 2]])) / pi[2]**2
        Jiy = (pi[2]*np.array([Mi[1, 0], Mi[1, 1], Mi[1, 2]]) \
              - pi[1]*np.array([Mi[2, 0], Mi[2, 1], Mi[2, 2]])) / pi[2]**2
        J_set.append(Jix)
        J_set.append(Jiy)

    for i in range(J.shape[0]):
        J[i] = J_set[i]
    return J

L'algorithme de Gauss-Newton est une technique d'optimisation itérative couramment utilisée pour résoudre des problèmes de moindres carrés non linéaires, tels que celui rencontré dans la triangulation non linéaire. Voici comment cela fonctionne :

Étant donné la fonction objective $E(X) = \sum_{i} \|e_i\|^2$ (où $e_i = x_i - P_iX$), notre objectif est de trouver les coordonnées 3D $X$ qui minimisent cette fonction.

1. **Initialisation** :
   - Initialiser $X$ avec une estimation initiale.
   - Définir un seuil de tolérance $\epsilon$ pour la convergence.

2. **Itération** :
   Pour chaque itération $k$ :

   a. **Calculer le Jacobien** :
      - Calculer la matrice jacobienne $J$ des dérivées partielles de $e_i$ par rapport à $X$ pour tous les $i$. Il s'agit d'une matrice $m \times n$, où $m$ est le nombre d'observations (points 2D) et $n$ est le nombre de paramètres (3 pour les coordonnées 3D).

   b. **Calculer les Résidus** :
      - Évaluer les résidus $e_i$ pour l'estimation actuelle de $X$.

   c. **Étape de Mise à Jour** :
      - Résoudre le système linéarisé $J^T J \delta X = -J^T e$ pour $\delta X$, où $J^T$ représente la transposée de la matrice jacobienne.
      - Mettre à jour l'estimation : $X \leftarrow X + \delta X$.

   d. **Vérification de Convergence** :
      - Si $\|\delta X\| < \epsilon$ ou si le changement dans la fonction objective est très faible, mettre fin aux itérations.

3. **Terminaison** :
   - Renvoyer l'estimation finale $X$.


In [5]:
image_data_dir = '../data/statue/'

image_paths = [os.path.join(image_data_dir, 'images', x) for x in sorted(os.listdir('../data/statue/images')) if '.jpg' in x]

In [6]:
matches_subset = np.load(os.path.join(image_data_dir,'matches_subset.npy'), allow_pickle=True,encoding='latin1')[0,:]
fundamental_matrices = np.load(os.path.join(image_data_dir,'fundamental_matrices.npy'),allow_pickle=True,encoding='latin1')[0,:]

In [7]:
im0 = cv2.imread(image_paths[0])
im_height, im_width, _ = im0.shape

* Nous considérons des paires de caméras séquentielles pour déterminer les matrices de caméras

$$M_1=K[I\mid 0]$$ et $$M_2=K[R \mid t]$$  où

$$K=\begin{bmatrix}
f & 0 &0 \\
 0&f  &0 \\
0 & 0 & 1
\end{bmatrix}$$

In [8]:

focal_length = 719.5459

K = np.eye(3)
K[0,0] = K[1,1] = focal_length


In [9]:

example_RT = np.array([[0.9736, -0.0988, -0.2056, 0.9994],
        [0.1019, 0.9948, 0.0045, -0.0089],
        [0.2041, -0.0254, 0.9786, 0.0331]])

In [10]:
camera_matrices = np.zeros((2, 3, 4))
camera_matrices[0, :, :] = K.dot(np.hstack((np.eye(3), np.zeros((3,1)))))
camera_matrices[1, :, :] = K.dot(example_RT)
image_points = matches_subset[0][:,0].reshape(2,2)

*  Pour le problème de triangulation avec N images, la solution des moindres carrés linéaires est la suivante
$$\delta_p=-(J^TJ)^{-1}J^Te$$
où
$$e=\begin{bmatrix}
e_1\\
\vdots\\
e_N
\end{bmatrix}=\begin{bmatrix}
p_1-M_n\hat{P}\\
\\
p_n-M_n\hat{P}
\end{bmatrix}$$
et
$$J=\begin{bmatrix}
\frac{\partial{e_1}}{\partial{\hat{P}_1}} &  \frac{\partial{e_1}}{\partial{\hat{P}_2}}&\frac{\partial{e_1}}{\partial{\hat{P}_3}} \\
 \vdots& \vdots & \vdots\\
\frac{\partial{e_N}}{\partial{\hat{P}_1}} &  \frac{\partial{e_N}}{\partial{\hat{P}_2}}&\frac{\partial{e_N}}{\partial{\hat{P}_3}}
\end{bmatrix}$$

In [11]:
P = linear_estimate_3d_point(image_points, camera_matrices)

In [12]:


for i in range(10):
        e = reprojection_error(P, image_points, camera_matrices)
        J = jacobian(P, camera_matrices)
        P -= np.linalg.inv(J.T.dot(J)).dot(J.T).dot(e)


* On peut ecrire le system comme suit
$$\begin{bmatrix}
e_1\\
e_2
\end{bmatrix}=\begin{bmatrix}
x-U/W\\
y-V/W
\end{bmatrix}$$
* où

$$\begin{bmatrix}
U\\
V\\
W
\end{bmatrix}=\begin{bmatrix}
m_{11} & m_{12} &m_{13}&m_{14} \\
m_{21} & m_{22} &m_{23}&m_{24} \\
m_{31} & m_{32} &m_{33}&m_{34}
\end{bmatrix}\begin{bmatrix}
X\\
Y\\
Z\\
1
\end{bmatrix}=\begin{bmatrix}
m_{11}X + m_{12}Y +m_{13} W+m_{14} \\
m_{21}X + m_{22}Y +m_{23}W+m_{24} \\
m_{31}X + m_{32}Y +m_{33}W+m_{34}
\end{bmatrix}$$
* La jacobian pour chaque point de correspondance est alors donné par

$$ [  \frac{\partial e_1}{\partial X}, \frac{\partial e_1}{\partial Y}, \frac{\partial e_1}{\partial Z}]=\frac{W [ m_{11},m_{12},m_{13}  ]-U [ m_{31},m_{32},m_{33} ]}{W^2}$$


$$ [  \frac{\partial e_2}{\partial X}, \frac{\partial e_2}{\partial Y}, \frac{\partial e_2}{\partial Z}]=\frac{W [ m_{21},m_{22},m_{23}  ]-V [ m_{31},m_{32},m_{33} ]}{W^2}$$




In [13]:

expected_3d_point = np.array([0.6774, -1.1029, 4.6621])
print("Difference: ", np.fabs(P - expected_3d_point).sum())

Difference:  0.002954993831610686
